In [1]:
class mine_sweeper:
    def __init__(self):
        self.np = __import__("numpy")
        self.pd = __import__("pandas")
        self.math = __import__("math")
        self.random = __import__("random")
        self.warnings = __import__("warnings")
        self.warnings.simplefilter('ignore')
        self.ipy = __import__("IPython")
    
    def starter(self):
        self.determine_board_size()
        
        self.mine_number = self.math.ceil((self.board_double) * 0.158)
        self.mine_index = tuple(self.random.sample(tuple(range(self.board_double)), self.mine_number))
        self.board = self.np.array(tuple(1 if i in self.mine_index else 0 for i in range(self.board_double))).reshape(self.board_size, -1)
        
        self.flag_set = set()
        self.open_list = []
        self.view()
        
        self.action()
    
    def determine_board_size(self):
        self.board_size_input = input("ボードサイズは？　　　")
        if self.board_size_input in tuple(str(i) for i in range(1, 301)):
            self.board_size = int(self.board_size_input)
            self.board_double = self.board_size ** 2
        else:
            self.error_print()
            return self.determine_board_size()
        
    def action(self):
        while True:
            print(f"残りFlag : {self.mine_number - len(self.flag_set)}")
            self.action_input = input("開ける場所を 行, 列　で指定してください。旗を立てる場合は F, 行, 列 と、消す場合は R, 行, 列 と書いてください。   ")
            self.action_flag = False
            self.action_remove_flag = False
            if self.action_input[0] in ("F", "f"):
                self.action_flag = True
            elif self.action_input[0] in ("R", "r"):
                self.action_remove_flag = True
            self.input_number_treat()
            if len(self.action_point) != 2:
                self.error_print()
                return self.action()
            
            if self.action_flag:
                if self.action_point in self.open_list:
                    self.error_print()
                    return self.action()
                self.flag_set.add(self.action_point)
                
            elif self.action_remove_flag:
                if not(self.action_point in self.flag_set):
                    self.error_print()
                    return self.action()
                self.flag_set.remove(self.action_point)
                
            else:
                if self.action_point in self.flag_set:
                    self.open_flag_input = input("旗のある場所を指定しています。構わず開ける場合は yes と書き、そうでない場合はそのまま enter してください。   ")
                    if not(self.open_flag_input in ("yes", "Yes", "YES", "y", "Y")):
                        return self.action()
                    else:
                        self.flag_set.remove(self.action_point)
                    
                self.open_list += [self.action_point]
                self.action_row, self.action_column = self.action_point
                if self.board[self.action_row, self.action_column] == 1:
                    print("Game over")
                    self.view(go = True)
                    break
                    
                self.action_open()
                
            self.view()
            if self.end:
                break
    
    def input_number_treat(self):
        self.action_input = list(self.action_input)
        self.action_input_blank = self.action_input + [" "]
        self.number_list = tuple(map(str, tuple(range(10))))
        self.action_point_list = []
        for i, x in enumerate(self.action_input):
            if x in self.number_list:
                num_str = x
                judge = True
                while judge:
                    judge = False
                    if i != len(self.action_input):
                        judge = self.action_input_blank[i + 1] in self.number_list
                    if not(judge):
                        break
                    num_str += self.action_input_blank[i + 1]
                    del self.action_input[i + 1]
                    del self.action_input_blank[i + 1]
                self.action_point_list += [int(num_str)]
        self.action_point = tuple(self.action_point_list)

    def view(self, go = False):
        self.pre_view_board = tuple(self.count_mine((i, j)) for i in range(self.board_size) for j in range(self.board_size))
        self.end = False
        if go:
            self.end = True
        if self.pre_view_board.count(-100) == self.mine_number:
            print("Clear !!")
            self.end = True
        self.index_board = tuple((i, j) for i in range(self.board_size) for j in range(self.board_size))
        self.pre_flagged_view_board = tuple("F" if i in self.flag_set else x for i, x in zip(self.index_board, self.pre_view_board))
        self.view_board = self.np.array(tuple("-" if i == -100 else i for i in self.pre_flagged_view_board)).reshape(self.board_size, -1)
        if self.end:
            self.view_board = self.np.array(tuple("M" if i in self.mine_index else x for i, x in enumerate(self.view_board.flatten()))).reshape(self.board_size, -1)
        self.view_df = self.pd.DataFrame(self.view_board)
        self.pd.set_option('display.max_rows', 300)
        self.pd.set_option('display.max_columns', 300)
        self.ipy.display.display(self.view_df)
    
    def count_mine(self, point):
        self.count_mine_row, self.count_mine_column = tuple(point)
        if not(point in self.open_list):
            return -100
        
        self.counter = 0
        for i in range(-1, 2):
            if (self.count_mine_row == 0 and i == -1) or (self.count_mine_row == self.board_size - 1 and i == 1):
                continue
            for j in range(-1, 2):
                if (self.count_mine_column == 0 and j == -1) or (self.count_mine_column == self.board_size - 1 and j == 1):
                    continue
                    
                if self.board[self.count_mine_row + i, self.count_mine_column + j] == 1:
                    self.counter += 1
                    
        return self.counter
    
    def action_open(self):        
        self.check_point_tuple = tuple((i, j) for i in range(-1, 2) for j in range(-1, 2))
        for open_point in self.open_list:
            
            if self.count_mine(open_point) != 0:
                continue
                
            for check_row, check_column in self.check_point_tuple:
                self.open_row, self.open_column = open_point
                self.check_point = (self.open_row + check_row, self.open_column + check_column)
                self.check_row, self.check_column = self.check_point
                
                self.check_point_set = set(self.check_point)
                self.over_edge_set = {-1, self.board_size}
                
                if self.check_point in self.open_list or not(self.check_point_set.isdisjoint(self.over_edge_set)):
                    continue
                
                if self.board[self.check_row, self.check_column] == 0:
                    self.open_list += [self.check_point]

    def error_print(self):
        print("入力が不正です。再度入力してください。")

In [2]:
sweeper = mine_sweeper()
sweeper.starter()

ボードサイズは？　　　5


,0,1,2,3,4
0,-,-,-,-,-
1,-,-,-,-,-
2,-,-,-,-,-
3,-,-,-,-,-
4,-,-,-,-,-


残りFlag : 4
開ける場所を 行, 列　で指定してください。旗を立てる場合は F, 行, 列 と、消す場合は R, 行, 列 と書いてください。
2,2


,0,1,2,3,4
0,-,-,-,-,-
1,-,1,1,1,1
2,-,1,0,0,0
3,-,1,1,1,1
4,-,-,-,-,-


残りFlag : 4
開ける場所を 行, 列　で指定してください。旗を立てる場合は F, 行, 列 と、消す場合は R, 行, 列 と書いてください。
4,2


...

,0,1,2,3,4
0,-,-,1,-,-
1,1,1,1,1,1
2,1,1,0,0,0
3,F,1,1,1,1
4,1,1,1,F,1


残りFlag : 2
開ける場所を 行, 列　で指定してください。旗を立てる場合は F, 行, 列 と、消す場合は R, 行, 列 と書いてください。
0,1


,0,1,2,3,4
0,-,1,1,-,-
1,1,1,1,1,1
2,1,1,0,0,0
3,F,1,1,1,1
4,1,1,1,F,1


残りFlag : 2
開ける場所を 行, 列　で指定してください。旗を立てる場合は F, 行, 列 と、消す場合は R, 行, 列 と書いてください。
0,4
Clear !!


,0,1,2,3,4
0,M,1,1,M,1
1,1,1,1,1,1
2,1,1,0,0,0
3,M,1,1,1,1
4,1,1,1,M,1
